In [2]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [5]:
# Assume the text we give as "Great product I love it"...
# So what we need to do ?
# Convert this into numerical then pass it to model to take a prediction

In [6]:
# Below consist remove_punctuation method and important dirs for preprocess our give text(what user gives)

In [3]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [4]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [5]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [6]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [7]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [8]:
# Make a dataframe and apply ourprevious text-preprocessing methods
def preprocessing(text):
    data = pd.DataFrame([text], columns=['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace(r'\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [9]:
# Convert that tweet(text) into numerical
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [21]:
# Test Case 1... Success
# txt = "awesome product. i love ths"
# preprocessed_text = preprocessing(txt)
# vectorized_text =  vectorizer(preprocessed_text, tokens)
# prediction = model.predict(vectorized_text)
# prediction

array([0])

In [22]:
# # Test Case 2... Success
# txt = "bad . i hat this"
# preprocessed_text = preprocessing(txt)
# vectorized_text =  vectorizer(preprocessed_text, tokens)
# prediction = model.predict(vectorized_text)
# prediction

array([1])

In [11]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [25]:
txt = "awesome product. i love it"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'

In [26]:
txt = "BAD, hate this"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'negative'